# 9.1. О чем этот модуль
В этом модуле мы рассмотрим классические методы рекомендательных систем:

- Ассоциативные правила.
- Коллаборативная фильтрация.
- Алгоритмы SVD и ALS.

# 9.2. Характеристики рекомендательных систем
## Задача рекомендательной системы

Проинформировать пользователя о продукте, который ему может быть наиболее интересен в данный момент времени.

## КОМУ НУЖНЫ РЕКОМЕНДАТЕЛЬНЫЕ СИСТЕМЫ?

→ клиенту — получает информацию о наиболее подходящих для него товарах, интересных мультимедийных продуктах и т. д. 

→ сервису — зарабатывает на предоставлении услуг.

## СТЕПЕНЬ ПЕРСОНАЛИЗАЦИИ

→ Неперсональные рекомендации — когда вам рекомендуют то же самое, что всем остальным. 

→ Персональные рекомендации используют всю доступную информацию о клиенте.

→ Более продвинутый вариант — рекомендации на данных из текущей сессии. Вы посмотрели несколько товаров, и внизу страницы вам предлагаются похожие.

# 9.3. Ассоциативные правила

Для того чтобы сравнить ассоциативные правила по их силе (значимости) необходимо ввести несколько метрик — Support, Confidence и Lift.

In [9]:
import numpy as np
import pandas as pd

In [14]:
a = np.array([[1,1,1,1],[2, 0, 0, 0],[3,1,1,0],[4,1,1,1],[5,0,1,1]])
df = pd.DataFrame(a, columns = ['tr', 'cola', 'beer', 'diapers'])
df

,tr,cola,beer,diapers
0,1,1,1,1
1,2,0,0,0
2,3,1,1,0
3,4,1,1,1
4,5,0,1,1


## SUPPORT
$$sup(x) = \frac{ \{ t \in T; X \in t \} }{|T|}$$

Здесь  — это X itemset, в котором находится items, T — количество транзакций.

Можно охарактеризовать этот показатель как индекс частоты встречаемости конкретного продукта в имеющихся транзакциях. Это для того случая, если нам интересен один конкретный продукт (item).

Чаще нам бывает важно, насколько часто какие-то два продукта встречаются вместе. Для такого случая мы будем рассчитывать следующий вариант показателя:

$$sup(x_1 \cup x_2) = \frac{ \sigma (x_1 \cup x_2) }{|T|}$$

Предположим, что  есть несколько транзакций, в которых присутствует пиво, подгузники и кола. Мы считаем количество совместных транзакций, в которых есть пиво и подгузники, и делим на общее количество транзакций.

        Получается, Support такого правила равен 3/5, или 60 %. 

In [24]:
df[(df.beer == 1)&(df.diapers == 1)][['beer', 'diapers']]

,beer,diapers
0,1,1
3,1,1
4,1,1


In [30]:
supp_b_and_d = df[(df.beer == 1)&(df.diapers == 1)].shape[0]/df.shape[0]
supp_b_and_d

0.6

## CONFIDENCE

Этот показатель высчитывается на основе метрики Support.

$$conf(x_1 \cup x_2) = \frac{ supp (x_1 \cup x_2) }{sup(T)}$$

Он определяет, как часто правило срабатывает для всего датасета.

У нас есть Support пива и подгузников, которое мы посчитали. Также мы можем посчитать Support только пива. 
        
        Отношение этих двух Support будет равно 3/4, или 75 %.

In [31]:
supp_b = df[(df.beer == 1)].shape[0]/df.shape[0]
supp_b

0.8

In [37]:
np.around(supp_b_and_d/supp_b, 2)

0.75

## LIFT 

Ещё одна метрика — Lift. Она вычисляется следующим образом:

- вычисляется Support совместной встречаемости двух продуктов;
- делится на произведение Support каждого из этих продуктов.

$$ lift(x_1 \cup x_2) = \frac{ supp (x_1 \cup x_2) }{supp(x_1)*supp(x_2)} $$


$$ lift = \frac{Confidence}{Expectedconfidence} = \frac{Подгузники|Пиво}{Подгузники} $$

In [38]:
supp_d = df[(df.diapers == 1)].shape[0]/df.shape[0]
supp_d

0.6

In [41]:
np.around(supp_b_and_d/(supp_d*supp_b), 2)

1.25

## АЛГОРИТМ APRIORI

Рассмотрим один из алгоритмов для построения ассоциативных правил — Apriori.

Apriori использует следующее утверждение: 

если X⊆Y, то supp(X) ≥ supp(Y).

Отсюда следуют два свойства:

- если Y встречается часто, то любое подмножество X: X⊆Y также встречается часто
- если X встречается редко, то любое супермножество Y: Y ⊇X также встречается редко

Apriori по уровням проходит по префиксному дереву и рассчитывает частоту X встречаемости подмножеств  в D. 

Таким образом:

- исключаются редкие подмножества и все их супермножества.
- рассчитывается supp(X) для каждого подходящего кандидата X размера k на уровне k.

## Задание 9.3.1

1. Какой показатель определяет частоту срабатываемости правила на нашем датасете?
- support
- confidence верно
- lift

2. Предположим, у нас есть данные о 300 транзакциях. Мы отметили, что в 62 из них покупают вместе форель и соевый соус. Рассчитайте показатель support, округлите до сотых (пример: 0.99).
- 0.21  верно 

3. Какие данные наиболее удобны для работы с ассоциативными правилами?
- Порядковые
- Бинарные верно
- Количественные
- Номинальные

## 9.4. Практика

In [42]:
import pandas as pd
df = pd.read_csv('data_fin.csv', sep=';')

Здесь есть ID для пользователя, ID для фильма и рейтинг, который поставил данный пользователь данному фильму.

In [45]:
df.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


Из полученного датасета возьмем только те записи, у которых наивысший рейтинг (5) и объединим их по "Cust_Id". Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с Id тех фильмов, которые ему понравились:

In [46]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [53]:
good.head()

Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

Сначала идёт ID пользователя, дальше через пробел фильмы, которые нравятся этому пользователю.

In [48]:
import apyori

Cделаем несколько ассоциативных правил. Мы можем регулировать их количество, меняя параметры алгоритмов. Посмотрим, какие ассоциативные правила получаются для support = 0.1.

In [75]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

Строго говоря, мы получили не сами ассоциативные правила, а генератор. Это можно проверить, если, например, вызвать переменную association_rules:

In [76]:
association_rules

<generator object apriori at 0x0000020328196AC8>

In [77]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [78]:
asr_df.sample(10)

,from,to,confidence,support,lift
1447,11521,14961 7230 9628,0.231988,0.0483051,4.33509
1631,16265,5582 7230 9628,0.398075,0.0476386,7.28915
731,11283,14240 7230,0.228238,0.0470303,2.12024
1984,14240,14961 7057 16265 9628 5582,0.204267,0.0408894,4.70847
1514,14240,14961 16265 5582,0.252971,0.0506389,4.59279
987,14240,14961 5582,0.310533,0.0621614,4.52132
1822,14240,14550 2452 7057 7230,0.24239,0.0485208,4.74881
99,11089,3962,0.657242,0.0875919,3.83536
1136,16265,2452 7230,0.453102,0.0542238,4.33267
1521,14240,2452 14961 1905,0.231647,0.0463703,4.7294


In [79]:
len(asr_df)

2023

Мы видим здесь таблицу, где 2023 ассоциативных правила, и для каждого рассчитаны известные нам показатели.

Для того чтобы перейти от Id фильмов, к их названиям, нужно загрузить еще один файл, в котором содержится Id фильма, год его производства и название:

In [80]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

Мы можем написать процедуру, которая будет выводить названия фильмов в ассоциативном правиле и фильм, которое это ассоциативное правило рекомендует:

In [81]:
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)

Можем посмотреть, как выглядит это правило. Например, вызовем сотое правило:

In [82]:
get_rule_title(asr_df.loc[99])

['Monsters']
----------->
['Finding Nemo (Widescreen)']


То есть, если человеку нравится фильм «Монстры», то мы советуем ему посмотреть фильм «В поисках Немо».

Перейдём к построению рекомендаций для случайного человека под id=159992. Посмотрим, какие фильмы он смотрел и как он их оценил. 

In [83]:
j = 159992
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

1144     The Wedding Planner
2151         What Women Want
5316       Miss Congeniality
6286            Pretty Woman
6385              Sister Act
7233            Men of Honor
10358          Runaway Bride
11148      Maid in Manhattan
13049       Two Weeks Notice
15581     Sweet Home Alabama
Name: Name, dtype: object

Как мы можем посчитать посчитать рекомендации для этого человека? Мы можем пройтись по всем правилам в нашей таблице и проверить, если они присутствуют в просмотрах человека и у них высокий рейтинг, значит это правило ему подходит и мы можем добавить этот фильм в список рекомендаций.

In [84]:
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Dirty Dancing', 'Pretty Woman'}


Сложность такой рекомендательной системы состоит в том, что мы ограничены теми ассоциативными правилами, которые были созданы. Если фильм редкий, то он в эти ассоциативные правила, скорее всего, не попадёт.

Также для разных людей будет разное количество рекомендаций, в зависимости от того, сколько фильмов посмотрел каждый из них. И, если нам захочется порекомендовать каждому пользователю минимум три фильма, то не с каждым пользователем мы сможем это сделать.

Эту проблему можно решить с помощью алгоритма Коллаборативная фильтрация. 

## Задание 9.4.1

1. Найдите фильмы, которые понравились пользователю с ID, равным 130. Понравившимися пользователю фильмами мы будем считать те фильмы, которым он поставил наивысшую оценку (5). Скопируйте все ID фильмов. Например: 68 943 325 1234.

In [91]:
good[good.index == 130].values

array(['1865 3456 3962 5515 6029 6428 8159 8327 8782 8784 11165 11242 11701 12084 12232 14482'],
      dtype=object)

In [94]:
get_rule_title(asr_df.loc[315])

['The Patriot']
----------->
['The Green Mile']


In [107]:
j = 21
result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
result_df = pd.DataFrame(result, columns = ['film'])
result_df['len'] = result_df.film.apply(lambda x: len(x))

In [115]:
result_df[result_df.len == result_df.len.min()]['film'].iloc[0]

'The Sixth Sense'

# 9.5. Коллаборативная фильтрация